In [2]:
import cv2
import os
from time import sleep
import shutil
import numpy as np
from PIL import Image

In [9]:
# Loading model
face_cascade_frontal = cv2.CascadeClassifier('data/models/haarcascade_frontalface_default.xml')

# Id to save info of the person
person_id = input('id to save user: ')

path = f'data/users/{person_id}'

if os.path.exists(path):
    shutil.rmtree(path)

os.makedirs(path)

id to save user:  luis_123


In [11]:
count_frame = 0
capture = cv2.VideoCapture(0)

while(count_frame < 30):

    ret, frame = capture.read()

    gray_scale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    location_face_frontal = face_cascade_frontal.detectMultiScale(gray_scale_frame, 1.2, 5)

    for (x,y,w,h) in location_face_frontal:

        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)     

        # Save current positive frame into data/users
        file_name = str(person_id) + "_" + str(count_frame) + ".jpg"
        full_path = os.path.join(path, file_name)
        
        cv2.imwrite(full_path, gray_scale_frame[y:y+h,x:x+w])

        cv2.imshow('Frame Capturing', frame)
        
        count_frame += 1
        
        sleep(0.25)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

In [12]:
model_recognizer = cv2.face.LBPHFaceRecognizer_create()

# function to get the images and label data
def LoadImageData(path):

    '''
    This functiion loads images into a list of numpy arrays and return the id retrieved
    from the image file name
    
    Parameters
    ----------
    
    path:str
        Path to find images
    
    Returns
    -------
    
    (list, np.array):tuple containing a list with numpy arrays from the images loaded and
    numpy array of the id respectively
    
    '''
    
    train_images_path = [os.path.join(path, image) for image in os.listdir(path)]
  
    train_images = []
    ids = []

    for image in train_images_path:

        image_matrix = Image.open(image)
        image_matrix = np.array(image_matrix)
        
        unique_id = os.path.split(image)[-1].split("_")[1]

        train_images.append(image_matrix)
        ids.append(int(unique_id))

    return train_images, np.array(ids)

images_data = os.listdir("data/users/")
print(f"Loading images to train - user: {images_data}")
train_data = []
ids = []

for folder in images_data:
    path = os.path.join("data/users", folder)
    t, i = LoadImageData(path)
    train_data += t
    ids += [i]

print("Training...")
model_recognizer.train(train_data, np.array(ids).reshape(-1,))

path_weights_results = "data/training/weights.yml"
print("Saving results into")
model_recognizer.write(path_weights_results)

Loading images to train - user: ['luis_123', 'clarisa_101']
Training...
Saving results into


In [18]:
font = cv2.FONT_HERSHEY_SIMPLEX

#Load users mapping
user_maps = {}
for user in images_data:
    name, number = user.split("_")
    user_maps[int(number)] = name.capitalize()


# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)

while True:

    ret, frame =cam.read()

    gray_scale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    location_face_frontal = face_cascade_frontal.detectMultiScale(gray_scale_frame,
                                                  scaleFactor=1.2, minNeighbors=5)

    for(x,y,w,h) in location_face_frontal:

        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

        user_id, confidence = model_recognizer.predict(gray_scale_frame[y:y+h, x:x+w])

        # Check if confidence is less than treshold, if confidence = 0, perfect match
        if (confidence < 70):
            user = names[user_id]
            confidence = f" {int(100 - confidence)}"
        else:
            user = "Unknown"
            confidence = f" {int(100 - confidence)}"
        
        cv2.putText(frame, str(user), (x+5, y-5), font, 1, (255,255,255), 2)
        cv2.putText(frame, str(confidence), (x+5, y+h-5), font, 1, (255,255,0), 1)  
    
    cv2.imshow('Face detection camera', frame) 

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()